# Cortex Analyst
### Analysing Structured Financial Data from the Marketplace
Review the stock information which is based on data you have gathered from the marketplace by running the following code below - NB - the code below is pointing to a preloaded table.  It is recomended to replace this table from the dataset from the marketplace.  You will find this table in the Snowflake Public Data (Paid) listing.

In [ ]:
SELECT * FROM ACCELERATE_AI_IN_FSI.DEFAULT_SCHEMA.STOCK_PRICE_TIMESERIES LIMIT 10;

## Analysing Market Place Share Price Trends the traditional way
A data Engineer may then want to pivot / transform the data to enable them to effectively create a report

In [ ]:
CREATE OR REPLACE TABLE DEFAULT_SCHEMA.STOCK_PRICES AS
SELECT 
    TICKER,
    ASSET_CLASS,
    PRIMARY_EXCHANGE_CODE,
    PRIMARY_EXCHANGE_NAME,
    DATE,
    "'All-Day High'" AS ALL_DAY_HIGH,
    "'All-Day Low'" AS ALL_DAY_LOW,
    "'Nasdaq Volume'" AS NASDAQ_VOLUME,
    "'Post-Market Close'" AS POST_MARKET_CLOSE,
    "'Pre-Market Open'" AS PRE_MARKET_OPEN,
    YEAR(DATE)::text AS YEAR,
    MONTHNAME(DATE) AS MONTHNAME, 
    MONTH(DATE) AS MONTHNO 
FROM (
    SELECT 
        TICKER,
        ASSET_CLASS,
        PRIMARY_EXCHANGE_CODE,
        PRIMARY_EXCHANGE_NAME,
        DATE,
        VARIABLE_NAME,
        VALUE
    FROM ACCELERATE_AI_IN_FSI.DEFAULT_SCHEMA.STOCK_PRICE_TIMESERIES
)
PIVOT (SUM(VALUE) FOR VARIABLE_NAME IN (ANY ORDER BY VARIABLE_NAME));

SELECT * FROM DEFAULT_SCHEMA.STOCK_PRICES WHERE TICKER = 'SNOW' LIMIT 5

The **Dashboard Designer** would then produce the visualisation which would meet the usecase requirements.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
default_ticker = 'SNOW'
st.markdown(f' #### {default_ticker} - STOCK MARKET PATTERNS')

sdate = st.number_input('Choose Year:',2020,2025,2025)
# Use the pivoted STOCK_PRICES table created in the previous cell
stock_table = session.table('DEFAULT_SCHEMA.STOCK_PRICES').filter(year(col('"DATE"'))==sdate)

def stock_metric(column_name, ticker):
    stocks = stock_table.filter(col('"TICKER"')==ticker)
    df = stocks.select(col('"DATE"'), col(f'"{column_name}"')).to_pandas()
    df.columns = ['DATE', 'VALUE']
    return st.line_chart(df, y='VALUE', x='DATE', color='#29B5E8')


ticker_df = stock_table.select(col('"TICKER"')).distinct().to_pandas()
ticker_df.columns = ['TICKER']
ticker_list = ticker_df['TICKER'].tolist()
default_index = ticker_list.index(default_ticker) if default_ticker in ticker_list else 0

ticker = st.selectbox('Select Ticker:', ticker_list, default_index)

col1,col2,col3 = st.columns(3)

with col1:
    st.markdown('#### ALL DAY HIGH')
    stock_metric('ALL_DAY_HIGH',ticker)
with col2:
    st.markdown('#### ALL DAY LOW')
    stock_metric('ALL_DAY_LOW',ticker)
with col3:
    st.markdown('#### NASDAQ VOLUME')
    stock_metric('NASDAQ_VOLUME',ticker)

col1,col2 = st.columns(2)
with col1:
    st.markdown('#### PRE MARKET OPEN')
    stock_metric('PRE_MARKET_OPEN',ticker)
with col2:
    st.markdown('#### POST MARKET CLOSE')
    stock_metric('POST_MARKET_CLOSE',ticker)

st.markdown('''You will see that you can get lots of information in a dashboard - streamlit makes this capabilty very flexible.

However, some users want to be able to answer ad-hoc questions without having to create a new
dashboard or waiting for a new improved dashboard to be built.  This is where **Cortex Analyst** comes into play

Plase go to to **Use Cortex Analyst to Explore the data** in the lab instructions to find out how **Cortex Analyst** works''')